In [14]:
import tensorflow as tf
import numpy as np
from  keras.utils import np_utils
from matplotlib import pyplot as plt


In [15]:
print(tf.__version__)

2.3.0


In [16]:
def generate_train_header(x, y, file='tain_data.h'):
    '''
    this method generate the
    :param x:  input x data size
    :param y:  input label (one hot label)
    :return:
    '''
    # quantize input x
    min_value = np.min(x)
    max_value = np.max(x)

    int_bits = int(np.ceil(np.log2(max(abs(min_value), abs(max_value)))))
    dec_bits = 7 - int_bits
    x = np.round(x*2**dec_bits).astype(np.int8)
    
#     print(x[0][0])
    data = x.astype(dtype="byte")
    label = y
    node = 0
    with open(file, 'w') as f:
        num_of_image = x.shape[0]
        
        print("Number of training images:",num_of_image)
        for i in range(num_of_image):
            f.write('#define NODE_%d_TRAIN_IMG_%d {'%(node,i) )
            (data[i]).flatten().tofile(f, sep=", ") # convert 0~1 to 0~127
            f.write('} \n')
            f.write('#define NODE_%d_TRAIN_IMG%d_LABEL'% (node, i))
            f.write(' %d \n \n' % label[i])
        f.write('#define NODE_%d_TOTAL_TRAIN_IMAGES %d \n \n'%(node,num_of_image))

        f.write('static q7_t NODE_%d_TRAIN_IMAGES[%d][%d] = {' % (node,num_of_image, data[0].flatten().shape[0]))
        f.write('NODE_%d_TRAIN_IMG_0'%(node))
        for i in range(num_of_image -1):
            f.write(',NODE_%d_TRAIN_IMG_%d'%(node, i+1))
        f.write('};\n\n')

        f.write('static q7_t NODE_%d_TRAIN_LABELS[%d] = {' % (node,num_of_image))
        f.write('NODE_%d_TRAIN_IMG0_LABEL'%(node))
        for i in range(num_of_image -1):
            f.write(',NODE_%d_TRAIN_IMG%d_LABEL'%(node, i+1))
        f.write('};\n\n')

#     return 000


In [17]:
def generate_test_bin(x, y, file='test_data.h'):
    '''
    method to generate the test data as a header file
    :param x:  input x data size
    :param y:  input label 
    :write to file 
    :return: nothing
    '''
    # quantize input x
    min_value = np.min(x)
    max_value = np.max(x)

    int_bits = int(np.ceil(np.log2(max(abs(min_value), abs(max_value)))))
    dec_bits = 7 - int_bits
    x = np.round(x*2**dec_bits).astype(np.int8)
    
    data = x
    label = y

    with open(file, 'w') as f:
        num_of_image = x.shape[0]
        print("Number of test images:",num_of_image)
        for i in range(num_of_image):
            f.write('#define TEST_IMG%d {'% (i))
            data[i].flatten().tofile(f, sep=", ")
            f.write('} \n')
            f.write('#define TEST_IMG%d_LABEL'% (i))
            f.write(' %d \n \n' % label[i])
        f.write('#define TOTAL_TEST_IMAGES %d \n \n'%(num_of_image))

        f.write('static q7_t TEST_IMAGES[%d][%d] = {' % (num_of_image, data[0].flatten().shape[0]))
        f.write('TEST_IMG0')
        for i in range(num_of_image -1):
            f.write(',TEST_IMG%d'%(i+1))
        f.write('};\n\n')

        f.write('static q7_t TEST_LABELS[%d] = {' % (num_of_image))
        f.write('TEST_IMG0_LABEL')
        for i in range(num_of_image -1):
            f.write(',TEST_IMG%d_LABEL'%(i+1))
        f.write('};\n\n')


In [21]:
## MNIST images


mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0




# subset_x_train = x_train[np.isin(y_train, [6,7,8]).flatten()]
# subset_y_train = y_train[np.isin(y_train, [6,7,8]).flatten()]-6

# subset_x_test = x_test[np.isin(y_test, [6,7,8]).flatten()]
# subset_y_test = y_test[np.isin(y_test, [6,7,8]).flatten()]-6


# indices = np.random.choice(500, n, replace=False)
# indices = np.random.choice(x_train.shape[0], 600, replace=False)
# print(indices)

generate_train_header(x_train[:700], y_train[:700], file='tain_data.h')
generate_test_bin(x_train[600:730], y_train[600:730], file='test_data.h')

# generate_test_bin(x_test, y_test, file='test_data.h')


# generate_train_header(subset_x_train, subset_y_train, file='cifar_train_data.h')
# subset_x_train.shape[0]
# federated_train_images(0, subset_x_train*127, subset_y_train, subset_x_train.shape[0], file='CIFAR3-TrainSet.h')

# subset_x_test.shape[0]

# imagesTestcfile(subset_x_test*127, subset_y_test,subset_x_test.shape[0] , file='CIFAR3-TestSet.h')


print ("done")

Number of training images: 700
Number of test images: 130
done


In [12]:
print(subset_x_train.shape)
print(subset_y_train.shape)
print (np.unique(y_train))

(48630, 28, 28)
(48630,)
[0 1 2 3 4 5 6 7 8 9]


In [11]:
!open .

In [ ]:
imgplot = plt.imshow(subset_x_train[21])
print(subset_y_train[21])